In [163]:
import pandas as pd
import altair as alt
import mysql.connector
import os
from dotenv import dotenv_values
import duckdb
import math
import numpy as np

In [168]:
df = pd.read_csv('lobbywatch_medienkonferenz.csv')
df_bezahlt = duckdb.query('select * from df where verguetung > 0').to_df()

# Untersuchte Interessenbindungen
Untersucht wurden alle Interessnbindungen, welche folgenden Kriterien entsprechen:
- Sind gemäss Parlamentariergesetz (ParlG) Art. 11 deklarationspflichtig
- Werden von einem gegenwärtig amtierenden Parlamentarier ausgeübt
- Es handelt sich hierbei nicht um eine hauptberufliche Tätigkeit
- Die Interessenbindung wurde gemäss dem Prozess von Lobbywatch recherchiert, kontrolliert und freigegeben
- Die Organisation ist keine Partei, parlamentarische Gruppe, parlamentarische Freundschaftsgruppe oder eine ausserparlamentarische Kommission

# Kennzahlen
Schauen wir uns zuerst ein paar Kennzahlen an

In [165]:
duckdb.query('''
select 'total' kennzahl, (select count(1) from df) wert, 100.0*(select count(1) from df)/(select count(1) from df) anteil
union
select 'deklariert', (select count(1) from df where status = 'deklariert'), 100.0*(select count(1) from df where status = 'deklariert')/(select count(1) from df)
union
select 'bezahlt', (select count(1) from df where verguetung > 0), 100.0*(select count(1) from df where verguetung > 0)/(select count(1) from df)
union
select 'mit angabe zur vergütung', (select count(1) from df where verguetung > 1), 100.0*(select count(1) from df where verguetung > 1)/(select count(1) from df)
union
select 'bezahlt aber undeklariert', (select count(1) from df where verguetung > 0 and status != 'deklariert'), 100.0*(select count(1) from df where verguetung > 0 and status != 'deklariert')/(select count(1) from df)
''').to_df()

,kennzahl,wert,anteil
0,total,2339,100.000000
1,deklariert,1749,74.775545
2,bezahlt,839,35.870030
3,mit angabe zur vergütung,254,10.859342
4,bezahlt aber undeklariert,45,1.923899


Bezahlte Interessenbindungen aufgeschlüsselt nach Branche.

In [169]:
source = duckdb.query('''
    select branche, count(1) count
    from df_bezahlt
    group by 1
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('branche', title='Branche'),
    tooltip='count'
)

alt.Chart(...)

In [199]:
source = duckdb.query('''
    select 'total' mass, fraktion, count(1) count
    from df_bezahlt
    group by 1, 2
    union all
    select 'pro parl.', fraktion, (select 1.0*count(1)/(select count(distinct parlamentarier) from df where fraktion = b.fraktion) from df_bezahlt where fraktion = b.fraktion)
    from df_bezahlt b
    group by 1, 2
''').to_df()

alt.Chart(source).mark_bar(opacity=1.0).encode(
    x=alt.X('count', title='Anzahl', stack=None),
    y=alt.Y('fraktion', title='Fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'])),
    color='mass',
    tooltip='count'
)

alt.Chart(...)

Hier sieht man wie sich die Fraktionen die bezahlten Mandate aufteilen innerhalb der Branchen.
Auffällig ist, dass die linken Parteien nur in der Umweltbranche eine Mehrheit der Lobbymandate haben (falls man die GLP zu den linken zählt). Hingegen sind sie in den Branchen Landwirtschaft, Verkehr und Wirtschaft sehr schwach vertreten.

In [178]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where 1=1
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 10)
    group by 1, 2
    order by branche, case fraktion when 'G' then 1 when 'S' then 2 when 'GL' then 3 when 'M-E' then 4 when 'RL' then 5 else 6 end
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', stack='normalize', axis=alt.Axis(format='%')),
    y=alt.Y('branche', title='Branche'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'], range=['#85B50C','#E4022D','#A0A000','#FF9100','#3A8BC1','#0A7228'])),
    order=alt.Order('color_fraktion_sort_index:Q'),
)

alt.Chart(...)

Umgekehrt kann man auch anschauen wo die Schwerpunkte der einzelnen Fraktionen liegen.
_Berücksichtigt hier sind alle Branchen mit insgesamt mindestens 20 bezahlten Lobbymandaten._

In [174]:
source = duckdb.query('''
    select branche, fraktion, count(1) count
    from df_bezahlt
    where verguetung>0
    and branche in (select branche from df_bezahlt group by branche having count(1) >= 20)
    group by 1, 2
    order by branche, case fraktion when 'G' then 1 when 'S' then 2 when 'GL' then 3 when 'M-E' then 4 when 'RL' then 5 else 6 end
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', axis=alt.Axis(format='%'), stack='normalize'),
    y=alt.Y('fraktion', title='Fraktion', sort=['G','S','GL','M-E','RL','V']),
    color='branche',
).properties(
    # height=400
)

alt.Chart(...)

In [156]:
alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anteil', stack='normalize', axis=alt.Axis(format='%')),
    y=alt.Y('branche', title='Branche'),
    color=alt.Color('fraktion', scale=alt.Scale(domain=['G','S','GL','M-E','RL','V'], range=['#85B50C','#E4022D','#A0A000','#FF9100','#3A8BC1','#0A7228'])),
    order=alt.Order('color_fraktion_sort_index:Q'),
)

alt.Chart(...)

In [7]:
source = duckdb.query('''
    select branche, partei, farbcode, count(1) count
    from df
    where partei in (select partei from df group by partei having count(1) >= 10)
    and branche in (select branche from df group by branche order by count(1) desc limit 9)
    group by 1, 2, 3
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('count', title='Anzahl'),
    y=alt.Y('partei', sort=['Grüne','SP','M','GLP','FDP','SVP'], title='Partei'),
    color=alt.Color('partei', scale=alt.Scale(domain=['Grüne','SP','M','GLP','FDP','SVP'], range=['#85B50C','#E4022D','#FF9100','#A0A000','#3A8BC1','#0A7228',])),
).properties(
    width=250
).facet(
    facet='branche',
    columns=3
)

alt.FacetChart(...)

In [8]:
source = duckdb.query('''
    select branche, partei, farbcode, count(1) count
    from df
    where 1=1
    and partei in (select partei from df group by partei having count(1) >= 10)
    and branche in (select branche from df group by branche order by count(1) desc limit 9)
    group by 1, 2, 3
''').to_df()

alt.Chart(source).mark_arc(innerRadius=40, stroke='white', strokeWidth=1.5, cornerRadius=4).encode(
    theta=alt.Theta('count', stack='normalize'),
    color=alt.Color('branche'),
    tooltip='count',
).properties(
    width=150,
    height=150
).facet(
    facet='partei',
    columns=3
)

alt.FacetChart(...)

In [210]:
source = duckdb.query('''
    select distinct p.branche, p.fraktion,
        count(id) over (partition by p.branche, p.fraktion) count,
        count(id) over (partition by p.branche) count_all
    from (
        select branche, fraktion
        from (select distinct fraktion from df_bezahlt) p
        cross join (select branche from df_bezahlt group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df_bezahlt on (p.branche, p.fraktion) = (df_bezahlt.branche, df_bezahlt.fraktion)
    where 1=1
''').to_df()

fraktion_dict = {
    'G': '#85B50C',
    'S': '#E4022D',
    'M-E': '#FF9100',
    'GL': '#A0A000',
    'RL': '#3A8BC1',
    'V': '#0A7228',
}
for p in dict(fraktion_dict).keys():
    if p not in source['fraktion'].values:
        del fraktion_dict[p]

fraktion_namen, fraktion_farben = list(fraktion_dict.keys()), list(fraktion_dict.values())

alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
).encode(
    theta=alt.Theta('fraktion', stack='center', scale=alt.Scale(domain=fraktion_namen, rangeMin=0)),
    radius=alt.Radius('rel_count:Q', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('fraktion', scale=alt.Scale(domain=fraktion_namen, range=fraktion_farben)),
    tooltip=[alt.Tooltip('fraktion'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='branche',
    columns=3
).resolve_scale(
    # radius='independent',
    radius='shared',
)

alt.FacetChart(...)

In [10]:
source = duckdb.query('''
    select distinct p.branche, p.partei,
        count(id) over (partition by p.branche, p.partei) count,
        count(id) over (partition by p.branche) count_all
    from (
        select branche, partei
        from (select partei from df group by partei having count(1) >= 10) p
        cross join (select branche from df group by branche having count(1) >= 20 /*order by count(1) desc limit 9*/) b
    ) p left join df on (p.branche, p.partei) = (df.branche, df.partei)
    where 1=1
''').to_df()

partei_dict = {
    'Grüne': '#85B50C',
    'SP': '#E4022D',
    'EVP': '#61B5A5',
    'M': '#FF9100',
    'GLP': '#A0A000',
    'FDP': '#3A8BC1',
    'SVP': '#0A7228',
}
for p in dict(partei_dict).keys():
    if p not in source['partei'].values:
        del partei_dict[p]

partei_namen, partei_farben = list(partei_dict.keys()), list(partei_dict.values())

alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    theta=alt.Theta('branche', stack='center'),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=['branche','count'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='partei',
    columns=3
).resolve_scale(
    # theta='shared',
    # radius='independent',
)

alt.FacetChart(...)

In [11]:
alt.Chart(source).mark_arc(
    innerRadius=0,
    # theta2=math.pi/4,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4
).encode(
    theta=alt.Theta('count', stack=None),
    radius=alt.Radius('branche:N', scale=alt.Scale(type='identity', rangeMin=10, rangeMax=70), stack='center'),
    color=alt.Color('branche'),
    tooltip=['branche','count'],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all'
).facet(
    facet='partei',
    columns=3
).resolve_scale(
    theta='independent',
    # radius='independent',
)

alt.FacetChart(...)

In [12]:
source = duckdb.query('''
    select distinct p.branche, p.parlamentarier, df.partei, df.kanton,
        count(id) over (partition by p.branche, p.parlamentarier) count,
        count(id) over (partition by p.branche) count_all
    from (
        select branche, parlamentarier
        from (select parlamentarier as parlamentarier from df group by 1 order by count(distinct id) desc limit 12) p
        cross join (select branche from df group by branche order by count(1) desc limit 8) b
    ) p left join df on (p.branche, p.parlamentarier) = (df.branche, df.parlamentarier)
    where 1=1
    and df.partei is not null and df.kanton is not null
''').to_df()

alt.Chart(source).mark_arc(
    innerRadius=0,
    stroke='white',
    strokeWidth=1.5,
    cornerRadius=4,
    # tooltip=True
).encode(
    theta=alt.Theta('branche', stack='center', scale=alt.Scale(rangeMin=0)),
    radius=alt.Radius('count', type='quantitative', scale=alt.Scale(type='sqrt', rangeMax=70), stack=False),
    color=alt.Color('branche'),
    tooltip=[alt.Tooltip('branche'), alt.Tooltip('count', title='anzahl'), alt.Tooltip('rel_count', type='quantitative', format='.1%', title='anteil')],
).properties(
    width=150,
    height=150
).transform_calculate(
    rel_count='datum.count / datum.count_all',
    parl_label='datum.parlamentarier + " (" + datum.partei + "/" + datum.kanton + ")"'
).facet(
    facet='parl_label:N',
    columns=3
).resolve_scale(
    # radius='independent',
    # radius='shared',
)

alt.FacetChart(...)

In [26]:
df

,branche,lobbygruppe,partei,farbcode,parlamentarier,id,organisation,art,funktion,rechtsform,verguetung
0,Aussenpolitik/Aussenwirtschaft,Aussenpolitik allgemein,SVP,#0A7D3A,Thomas Aeschi,11388,Parlamentarischer Verein Schweiz - USA,vorstand,vizepraesident,Verein,1
1,Aussenpolitik/Aussenwirtschaft,EU-Gegner,SVP,#0A7D3A,Jean-Luc Addor,2434,proTell Gesellschaft für ein freiheitliches Wa...,vorstand,praesident,Verein,1
2,Aussenpolitik/Aussenwirtschaft,Hilfswerke international,SP,#FF0505,Claudia Friedl,2266,"SWISSAID, Schweizerische Stiftung für Entwickl...",vorstand,mitglied,Stiftung,600
3,Aussenpolitik/Aussenwirtschaft,Hilfswerke international,SP,#FF0505,Carlo Sommaruga,6005,Solidar Suisse,vorstand,praesident,Verein,15000
4,Bildung,Bildung/Wissenschaft,FDP,#0A4BD6,Andri Silberschmidt,11912,"FH SCHWEIZ, Dachverband der Absolventinnen und...",vorstand,praesident,Verein,1
...,...,...,...,...,...,...,...,...,...,...,...
906,Wirtschaft,Versicherungen,SVP,#0A7D3A,Therese Schläpfer,10844,Groupe Mutuel Holding SA,beirat,mitglied,AG,1
907,Wirtschaft,Wirtschaft allgemein,FDP,#0A4BD6,Martin Schmid,1335,economiesuisse,vorstand,mitglied,Verein,1
908,Wirtschaft,Wirtschaft allgemein,M,#FF9B03,Pirmin Bischof,1810,W.A. de Vigier Stiftung,vorstand,mitglied,Stiftung,1
909,Wirtschaft,Wirtschaft allgemein,M,#FF9B03,Elisabeth Schneider-Schneiter,4613,Handelskammer beider Basel,vorstand,praesident,Verein,30000


In [41]:
duckdb.query('''
select fraktion, branche, (select count(1) from df where branche = b.branche and fraktion = f.fraktion) count
from (select distinct fraktion from df) as f
cross join (select distinct branche from df) as b
group by 1, 2
''').to_df()

alt.Chart(source).mark_rect().encode(
    x='branche',
    y='fraktion',
    color=alt.Color('count',
        scale=alt.Scale(scheme='yelloworangered', domain=[0,110]),
        legend=alt.Legend(direction='horizontal')
    )
)

alt.Chart(...)

In [196]:
duckdb.query('''
select count(distinct parlamentarier) from df_bezahlt
''').to_df()

,count(DISTINCT parlamentarier)
0,204
